In [ ]:
%load_ext autoreload

In [ ]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partialnew_region2
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
from pydrake.all import ClpSolver

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, HPolyhedron
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram
import meshcat
from pydrake.all import MosekSolver
from pydrake.all import SolutionResult


# Build plant

In [ ]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [ ]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)

# Build Certified Iris Region Object

In [ ]:
iris_kwargs = {
    'iris_starting_ellipse_vol': 1e-5,
    'iris_plane_pullback': 1e-4,
    'iris_max_faces': -1
}

In [ ]:
iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)

In [ ]:
seed_points = np.tan(np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ])    
                        /2)

regions, ellipses = iris_generator.iris_in_rational_space(seed_points)


In [ ]:
trajectory_start = np.tan(np.array([-1, -2, 0.5])/2) # seed_points[1,:]
trajectory_end = np.tan(np.array([0.25, -1.6, -0.25])/2)  #seed_points[3,:]

In [ ]:
region_num = 0
num_var = regions[region_num].A().shape[1]

A_red = np.vstack([np.eye(num_var), -np.eye(num_var)])
fact = 0.25
b_red = np.hstack([fact*np.ones(num_var), fact*np.ones(num_var)])

region = HPolyhedron(np.vstack([regions[region_num].A(), A_red]), 
                     np.concatenate([regions[region_num].b(), b_red]))
# region = HPolyhedron(A_red, b_red)

num_faces = region.A().shape[0]
solver = MosekSolver()

In [ ]:
def prune_region(region):
    num_faces = region.A().shape[0]
    solver = MosekSolver()
    kept_indices = np.arange(num_faces).tolist()
    excluded_indices = []
    
    for excluded_index in range(num_faces):
        prog = MathematicalProgram()
        x = prog.NewContinuousVariables(region.A().shape[1], 'x')
        cur_kept_indices = kept_indices.copy()
        cur_kept_indices.remove(excluded_index)

        c_to_check_redundant = region.A()[excluded_index,:][np.newaxis, :]
        d_to_check_redundant = np.atleast_2d(region.b()).T[excluded_index,:][np.newaxis, :]

        A_to_check_against = region.A()[cur_kept_indices,:]
        b_to_check_against = region.b()[cur_kept_indices]

        prog.AddLinearCost(-c_to_check_redundant.T, 0., x)
        prog.AddLinearConstraint(A_to_check_against, -np.inf*np.ones_like(b_to_check_against),
                                 b_to_check_against, x)
        prog.AddLinearConstraint(c_to_check_redundant, -np.inf*np.ones_like(d_to_check_redundant), 
                                 d_to_check_redundant+1, x)

        solution = solver.Solve(prog)
        if solution.get_solution_result() != SolutionResult.kSolutionFound:
            raise ValueError(f"Solution not found status is {solution.get_solution_result()}. Polytope might be empty")
        opt_val = -solution.get_optimal_cost()
        if opt_val <= d_to_check_redundant:
            excluded_indices.append(excluded_index)
            kept_indices.remove(excluded_index)
    new_region = HPolyhedron(region.A()[kept_indices, :], region.b()[kept_indices])
    return new_region, kept_indices, excluded_indices
    
    

In [ ]:
A = np.vstack([np.eye(3), -np.eye(3), np.eye(3), -np.eye(3)])
b = np.concatenate([np.ones(6), 0.25*np.ones(6)])
new_region, kept_indices, excluded_indices = prune_region(HPolyhedron(A,b))
print(f"kept_indices = {kept_indices}")
print(f"excluded_indices = {excluded_indices}")

In [ ]:
for region in regions:
    new_region, kept_indices, excluded_indices = prune_region(region)
#     print(np.hstack([region.A(), region.b()[:, np.newaxis]]))
#     print()
#     print(np.hstack([new_region.A(), new_region.b()[:, np.newaxis]]))
    print(excluded_indices)
    print()

In [ ]:
new_region, kept_indices, excluded_indices = prune_region(region)


In [ ]:
print(region.A())
print()
print(new_region.A())

In [ ]:
print(excluded_indices)